## Problema rellenar una mochila sin valores

In [11]:
import numpy as np
import torch
import tensorkrowch as tk

In [34]:
def tensor_generator(pesos:np.array, valores:np.array, n_elementos:np.array, capacidad:int, tao:float, lambda1:float):
    lista_de_tensores = []
    n_clases = len(pesos)
    tensor = tensor_initial_generator(pesos[0],valores[0],n_elementos[0],capacidad,0,tao)

    lista_de_tensores.append(tensor)

    for n in range(1, n_clases-1):
        tensor = tensor_intermediate_generator(pesos[n], valores[n], n_elementos[n], capacidad, tensor.shape[1],tao)
        lista_de_tensores.append(tensor)

    tensor = tensor_final_generator(pesos[-1], valores[-1], n_elementos[-1], capacidad, tensor.shape[1],tao, lambda1)
    lista_de_tensores.append(tensor)

    return lista_de_tensores



In [14]:
def tensor_initial_generator(tn:tk.TensorNetwork, peso:int, valor:int, n_elementos:int, capacidad:int, previous_weight_solution:int,tao)->np.array:
    """
    Funcion que genera el tensor inicial.
    """
    tamaño_1 = min((capacidad-previous_weight_solution)//peso,n_elementos)+1
    tamaño_2 = min(capacidad, (tamaño_1-1)*peso+previous_weight_solution)+1
    tensor = np.zeros((tamaño_1,tamaño_2))
    for i in range(tamaño_1):
        elemento = i*peso + previous_weight_solution
        tensor[i][elemento] = np.exp(tao*valor*i)
    
    node = tk.Node(tensor = torch.from_numpy(tensor), network = tn, name = 'Nodo(0)', axes_names = ['up', 'down'])
    return node,tensor

In [15]:
def tensor_intermediate_generator(tn:tk.TensorNetwork, peso:int, valor:int, n_elementos:int, capacidad:int, previous_weight:int,tao:float,value)->np.array:

    tamaño_1 = previous_weight
    tamaño_2 = min(capacidad, n_elementos*peso+previous_weight-1)+1
    tensor = np.zeros((tamaño_1,tamaño_2))
    for i in range(tamaño_1):
        n_elementos_restantes = min((capacidad -i)//peso,n_elementos)+1
        for  j in range(n_elementos_restantes):
            elemento = i + j*peso
            tensor[i][elemento] = np.exp(tao*valor*j)
    node = tk.Node(tensor = torch.from_numpy(tensor), network = tn, name  = f'Nodo({value})', axes_names = ['up',  'down'])
    return node,tensor

In [16]:
def tensor_final_generator(tn:tk.TensorNetwork, peso:int, valor:int,n_elementos: int, capacidad: int, previous_weight:int, tao:float, lambda1:float)->np.array:
    tamaño_1 = previous_weight
    tensor = np.zeros((tamaño_1))
    for i in range(tamaño_1):
        n_elementos_posibles = min((capacidad -i)//peso,n_elementos)+1
        for j in range(n_elementos_posibles):
            elemento = i +j*peso
            tensor[i] += np.exp(tao*j*valor)
            #tensor[i] += np.exp(-tao*(capacidad - elemento))
            #tensor[i] += np.exp(-lambda1*(capacidad - elemento))
    node = tk.Node(tensor = torch.from_numpy(tensor), network = tn, name  = 'Nodo(f)', axes_names = ['up'])
    return node, tensor

In [56]:
def generate_tensor_network(pesos:np.array, valores:np.array, n_elementos:np.array, capacidad:int, tao:float, lambda1:float):
    n_clases = len(pesos)
    lista_de_tensores = []
    tn = tk.TensorNetwork()
    node,tensor = tensor_initial_generator(tn,pesos[0],valores[0],n_elementos[0],capacidad,0,tao)
    lista_de_tensores.append(node)
    for n in range(1,n_clases-1):
        node,tensor = tensor_intermediate_generator(tn,pesos[n], valores[n], n_elementos[n], capacidad, tensor.shape[1],tao,n)
        lista_de_tensores.append(node)

    node,tensor = tensor_final_generator(tn,pesos[-1], valores[-1], n_elementos[-1], capacidad, tensor.shape[1],tao, lambda1)
    lista_de_tensores.append(node)

    return lista_de_tensores,tn


def connect_tensor_network(lista_de_tensores: list):
    n_clases = len(lista_de_tensores)
    for n in range(0, n_clases-1):
        lista_de_tensores[n]['down']^lista_de_tensores[n+1]['up']
    return


        

In [18]:
def tensor_contraction(lista_de_tensores:list):

    n_tensores = len(lista_de_tensores)
    tensores_intermedios = []
    vector = lista_de_tensores[-1]
    tensores_intermedios.append(vector)
    for j in range(n_tensores-1,0,-1):
        vector = lista_de_tensores[j-1] @ vector
        #if max(vector) > 1e200:
            #vector = vector*1e-25
        tensores_intermedios.append(vector)
    tensores_intermedios.reverse()
    return vector,tensores_intermedios


In [ ]:
def solver(pesos:np.array, valores:np.array, n_elementos:np.array, capacidad:int, tao:float, lambda1:float)->np.array:
    """
    Funcion que resuelve el problema de la mochila sin valores.
    """ 
    n_clases = len(pesos)
    solution = np.zeros(n_clases)
    #lista_de_tensores = tensor_generator(pesos, valores, n_elementos, capacidad, tao, lambda1)
    lista_de_tensores,tn = generate_tensor_network(pesos, valores, n_elementos, capacidad, tao, lambda1)
    connect_tensor_network(lista_de_tensores)
    vector_salida, tensores_intermedios = tensor_contraction(lista_de_tensores)


    max = np.max(tensores_intermedios[0])
    solution[0] = np.argmax(vector_salida.tensor)


    for n in range(1,n_clases-1):
        cuenta_peso = int(np.dot(solution[:n],pesos[:n]))
        new_node1, new_initial_tensor = tensor_initial_generator(tn,pesos[n], valores[n], n_elementos[n], capacidad, cuenta_peso,tao)
        aux_node = tk.Node(tensor = tensores_intermedios[n+1].tensor[:new_initial_tensor.shape[1]], network = tn, name  = 'Nodo_aux', axes_names = ['up'])
        new_node1['down']^aux_node['up']
        result = new_node1 @ aux_node
        solution[n]= np.argmax(result.tensor)

    cuenta_peso = np.dot(solution[:-1],pesos[:-1])
    solution[-1] = min((capacidad-cuenta_peso)//pesos[-1],n_elementos[-1])
    print("La solucion es: ", solution)
    print("El peso total es: ", np.dot(solution,pesos))
    print("El valor total es: ", np.dot(solution,valores))
    return np.dot(solution,pesos),np.dot(solution,valores),max

In [62]:
clases = 100
capacidad = 345
np.random.seed(1)
pesos = np.random.randint(1,10,clases)
valores = np.random.rand(clases)
n_elementos = [5]*clases


tao = 10
lambda1 = 1

b,c,g=solver(pesos,valores, n_elementos, capacidad, tao, lambda1)


La solucion es:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 0. 0. 0.
 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 0. 5. 5. 5. 5. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 5. 5. 0. 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 5. 5.
 0. 5. 5. 0. 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 5. 5. 3. 0. 0. 0. 0. 0. 5. 0.
 0. 5. 5. 5.]
El peso total es:  345.0
El valor total es:  69.50501119801989


In [60]:

def mochila_greedy(pesos, valores, capacidad, n_elementos):
    n_clases = len(pesos)
    
    # Calculamos la relación valor/peso para cada clase
    valor_peso = valores / pesos
    
    # Ordenamos los elementos por su valor/peso de mayor a menor
    indices_ordenados = np.argsort(valor_peso)[::-1]  # Orden descendente
    
    # Inicializamos variables
    peso_actual = 0
    valor_actual = 0
    solucion = np.zeros(n_clases, dtype=int)
    
    # Recorremos los objetos en orden greedy
    for i in indices_ordenados:
        # Tomamos la mayor cantidad posible de este objeto sin exceder la capacidad
        cantidad = min(n_elementos[i], (capacidad - peso_actual) // pesos[i])
        solucion[i] = cantidad
        peso_actual += cantidad * pesos[i]
        valor_actual += cantidad * valores[i]
        
        # Si llenamos la capacidad, salimos del bucle
        if peso_actual >= capacidad:
            break
    
    return solucion, valor_actual

# Ejecutamos el algoritmo

mejor_solucion, mejor_valor = mochila_greedy(pesos, valores, capacidad, n_elementos)

print("Mejor solución encontrada (greedy):", mejor_solucion)
print("Valor total de la mejor solución (greedy):", mejor_valor)
print("Peso total de la mejor solución (greedy):", np.sum(mejor_solucion * pesos))
#print(b==mejor_valor)

Mejor solución encontrada (greedy): [0 0 0 5 5 5 0 0 0 5 0 5 0 5 0 0 0 0 0 5 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 5 0 5 5 0 0 0 0 0 0 0 0 0 5 5 0 0 0 5 0 0 0 0 0 0 0 0 5 0 5 5 0 2
 5 0 0 0 0 0 0 0 0 0 0 0 5 5 1 0 0 0 0 0 5 0 0 5 5 5]
Valor total de la mejor solución (greedy): 79.08540382349945
Peso total de la mejor solución (greedy): 345
